# Task 2: Lookalike Model

**Import necessary libraries**

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

**File paths**

In [2]:
customer_file = '/kaggle/input/dataset3/Customers.csv'
product_file = '/kaggle/input/dataset3/Products.csv'
transaction_file = '/kaggle/input/dataset3/Transactions.csv'

**Load the datasets**

In [3]:
customers = pd.read_csv(customer_file)
products = pd.read_csv(product_file)
transactions = pd.read_csv(transaction_file)

**Merge datasets**

In [4]:
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')